## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.06,49,20,17.27,few clouds
1,1,Hamilton,US,39.1834,-84.5333,81.41,81,28,1.01,light rain
2,2,Teya,RU,60.3778,92.6267,57.99,77,100,2.04,overcast clouds
3,3,Puerto Varas,CL,-41.3195,-72.9854,53.82,66,0,6.91,clear sky
4,4,Matara,LK,5.9485,80.5353,78.46,86,100,9.69,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum temperature preference? "))
max_temp = float(input("What is your maximum temperature preference? "))

What is your minimum temperature preference? 70
What is your maximum temperature preference? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hamilton,US,39.1834,-84.5333,81.41,81,28,1.01,light rain
4,4,Matara,LK,5.9485,80.5353,78.46,86,100,9.69,overcast clouds
6,6,Lafia,NG,8.4833,8.5167,72.39,96,100,4.36,overcast clouds
7,7,Padrauna,IN,26.9025,83.9819,78.67,92,100,2.01,heavy intensity rain
9,9,Nicoya,CR,10.1483,-85.4520,89.15,77,98,6.78,overcast clouds
11,11,San Cristobal,VE,7.7669,-72.2250,84.38,80,68,7.29,broken clouds
18,18,Cockburn Town,TC,21.4612,-71.1419,85.96,70,40,12.66,scattered clouds
19,19,Nurota,UZ,40.5614,65.6886,73.49,15,0,6.80,clear sky
25,25,Sao Filipe,CV,14.8961,-24.4956,76.03,76,11,8.03,few clouds
27,27,Dobrinje,BA,44.0497,18.1194,86.74,57,67,4.59,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                93
City                   93
Country                92
Lat                    93
Lng                    93
Max Temp               93
Humidity               93
Cloudiness             93
Wind Speed             93
Current Description    93
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID                92
City                   92
Country                92
Lat                    92
Lng                    92
Max Temp               92
Humidity               92
Cloudiness             92
Wind Speed             92
Current Description    92
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hamilton,US,81.41,light rain,39.1834,-84.5333,
4,Matara,LK,78.46,overcast clouds,5.9485,80.5353,
6,Lafia,NG,72.39,overcast clouds,8.4833,8.5167,
7,Padrauna,IN,78.67,heavy intensity rain,26.9025,83.9819,
9,Nicoya,CR,89.15,overcast clouds,10.1483,-85.4520,
11,San Cristobal,VE,84.38,broken clouds,7.7669,-72.2250,
18,Cockburn Town,TC,85.96,scattered clouds,21.4612,-71.1419,
19,Nurota,UZ,73.49,clear sky,40.5614,65.6886,
25,Sao Filipe,CV,76.03,few clouds,14.8961,-24.4956,
27,Dobrinje,BA,86.74,broken clouds,44.0497,18.1194,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# "Pandas will recognise a value as null if it is a np.nan object, 
# which will print as NaN in the DataFrame. Your missing values are probably empty strings, 
# which Pandas doesn't recognise as null. To fix this, 
# you can convert the empty stings (or whatever is in your empty cells) 
# to np.nan objects using replace(), 
# and then call dropna()on your DataFrame to delete rows with null tenants."
# source: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe/56708633
# Add numpy dependency above

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} 'F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))